<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/notebooks/01_sh_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%time
data=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q2.xlsx')
#data=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2019.xlsx')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [3]:
data.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,EMPLOYER_NAME,TRADE_NAME_DBA,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,NAICS_CODE,EMPLOYER_POC_LAST_NAME,EMPLOYER_POC_FIRST_NAME,EMPLOYER_POC_MIDDLE_NAME,EMPLOYER_POC_JOB_TITLE,EMPLOYER_POC_ADDRESS1,EMPLOYER_POC_ADDRESS2,EMPLOYER_POC_CITY,EMPLOYER_POC_STATE,EMPLOYER_POC_POSTAL_CODE,...,AGENT_ATTORNEY_PHONE,AGENT_ATTORNEY_PHONE_EXT,AGENT_ATTORNEY_EMAIL_ADDRESS,LAWFIRM_NAME_BUSINESS_NAME,STATE_OF_HIGHEST_COURT,NAME_OF_HIGHEST_STATE_COURT,WORKSITE_WORKERS,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_ADDRESS1,WORKSITE_ADDRESS2,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_TRACKING_NUMBER,PW_WAGE_LEVEL,PW_OES_YEAR,PW_OTHER_SOURCE,PW_OTHER_YEAR,PW_SURVEY_PUBLISHER,PW_SURVEY_NAME,TOTAL_WORKSITE_LOCATIONS,AGREE_TO_LC_STATEMENT,H-1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL
0,I-200-19357-214767,Certified,2019-12-23,2020-01-01,NaT,H-1B,Senior Systems Analyst JC60,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,CTSUS,211 QUALITY CIRCLE,NaN,COLLEGE STATION,TX,77845,UNITED STATES OF AMERICA,NaN,19793296682,NaN,541512,CHANDRASEKARAN,SABARISH,NaN,ASSOCIATE DIRECTOR,211 QUALITY CIRCLE,NaN,COLLEGE STATION,TX,77845,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Y,Computershare Inc,"480, Washington Boulevard, Floor 26",NaN,Jersey City,HUDSON,NJ,7310,92435.00,NaN,Year,92435.00,Year,NaN,II,7/1/2019 - 6/30/2020,NaN,NaN,NaN,NaN,1.0,Y,Y,N,Y,WAGE,NaN,Disclose Business,NaN,NaN,NaN,NaN,lca.immigration@cognizant.com
1,I-200-19357-214772,Certified,2019-12-23,2020-01-01,NaT,H-1B,Hardware Engineer,17-2072.00,"Electronics Engineers, Except Computer",Y,2020-06-16,2023-06-15,30,5,5,5,5,5,5,"SYNOPSYS, INC.",NaN,690 E. Middlefield Rd,NaN,Mountain View,CA,94043,UNITED STATES OF AMERICA,NaN,16505845000,NaN,541512,Nguyen,Beth,NaN,HUMAN RESOURCES MANAGER,690 EAST MIDDLEFIELD RD.,NaN,Mountain View,CA,94043,...,1.213625e+10,NaN,FLAG-LA@tsmimmigration.com,Tafapolsky Smith Mehlman LLP,CA,Supreme Court,30.0,N,NaN,690 East Middlefield Road,NaN,Mountain View,SANTA CLARA,CA,94043,113684.00,188684.0,Year,113684.00,Year,NaN,NaN,NaN,Survey,2019.0,Radford,Global Technology Survey (Hardware Development...,1.0,Y,N,N,NaN,NaN,NaN,Disclose Business,Zanetti,Lauren,B,Tafapolsky Smith Mehlman LLP,lauren.zanetti@tsmimmigration.com
2,I-200-19357-214781,Certified,2019-12-23,2020-01-01,NaT,H-1B,Senior Systems Analyst JC60,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,CTSUS,211 QUALITY CIRCLE,NaN,COLLEGE STATION,TX,77845,UNITED STATES OF AMERICA,NaN,19793296682,NaN,541512,CHANDRASEKARAN,SABARISH,NaN,ASSOCIATE DIRECTOR,211 QUALITY CIRCLE,NaN,COLLEGE STATION,TX,77845,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Y,"Fiserv, Inc.",5565 Glenridge Connector NE,NaN,Atlanta,FULTON,GA,30342,77771.00,NaN,Year,77771.00,Year,NaN,II,7/1/2019 - 6/30/2020,NaN,NaN,NaN,NaN,1.0,Y,Y,N,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,lca.immigration@cognizant.com
3,I-200-19357-214799,Certified,2019-12-23,2020-01-01,NaT,H-1B,Senior Systems Analyst JC60,15-1121.00,Computer Systems Analysts,Y,2020-01-01,2022-12-31,1,0,1,0,0,0,0,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,CTSUS,211 QUALITY CIRCLE,NaN,COLLEGE STATION,TX,77845,UNITED STATES OF AMERICA,NaN,19793296682,NaN,541512,CHANDRASEKARAN,SABARISH,NaN,ASSOCIATE DIRECTOR,211 QUALITY CIRCLE,NaN,CO

In [4]:
#create a copy of input data to avoid reimporting it in case the working copy gets corrupted. 
data_copy=data.copy()

In [5]:
#Dictionary to store notes on preprocessing steps
dict_notes={}

In [59]:
#store feature statictics in a dataframe
df_data_statistics=pd.DataFrame(data=None,index=data.columns,columns=['Dtype','percent_missing','cardinality','preprocess_action','preprocess_comment','new_feature_name','new_feature_logic'])

In [7]:
#write reusable function to display
#record count per class
print('Count per class:',data.groupby(['CASE_STATUS']).size())
#df shape - records and features
print('Feature shape:', data.shape)

Count per class: CASE_STATUS
Certified                150011
Certified - Withdrawn      2884
Denied                     1186
Withdrawn                  3092
dtype: int64
Feature shape: (157173, 96)


In [8]:
#add observation
dict_notes['output_class']='Unbalanced classes'

In [60]:
#dataframe statistics
for column in data.columns:
  #identify numeric, non-numeric and date columns
  df_data_statistics.Dtype.loc[column]=data[column].dtype
  #% missing data for each column
  df_data_statistics.percent_missing.loc[column]=(data.shape[0]-data[column].count())*100/data.shape[0]
  #Cardinality of each column
  df_data_statistics.cardinality.loc[column]=(data.shape[0]-len(data[column].unique()))*100/data.shape[0]

In [61]:
#drop features with missing values >50%
missing_threshold=40.0
for idx in df_data_statistics[df_data_statistics.percent_missing>=missing_threshold].index:
  df_data_statistics.loc[[idx],['preprocess_action','preprocess_comment']]=['Drop column','missing values>='+str(missing_threshold)+'% of total']

In [62]:
#drop features with high cardinality
cardinality_threshold=80.0
for idx in df_data_statistics[df_data_statistics.cardinality<80.0].index:
  df_data_statistics.loc[[idx],['preprocess_action','preprocess_comment']]=['Drop column','High Cardinality, threshold '+str(cardinality_threshold)+'% of total']

In [63]:
#Separate target column
df_data_statistics.loc[['CASE_STATUS'],['preprocess_action','preprocess_comment']]=['Pop column into a separate list','Target feature']

In [65]:
#FEATURE Engineering - date columns
print(list(df_data_statistics[df_data_statistics.Dtype=='<M8[ns]'].index))
#Create a new feature - PROCESSING_DAYS from 'RECEIVED_DATE', 'DECISION_DATE'
df_data_statistics.loc[['RECEIVED_DATE', 'DECISION_DATE'],['preprocess_action','preprocess_comment','new_feature_name','new_feature_logic']]=['Drop column','Feature engineering','PROCESSING_DAYS','days(DECISION_DATE-RECEIVED_DATE)']
#Create a new feature - VALIDITY_DAYS from 'BEGIN_DATE', 'END_DATE'
df_data_statistics.loc[['BEGIN_DATE', 'END_DATE'],['preprocess_action','preprocess_comment','new_feature_name','new_feature_logic']]=['Drop column','Feature engineering','VALIDITY_DAYS','days(END_DATE-BEGIN_DATE)']

['RECEIVED_DATE', 'DECISION_DATE', 'ORIGINAL_CERT_DATE', 'BEGIN_DATE', 'END_DATE']


In [68]:
#Feature engineering - split SOC_CODE into 2 new features - SOC_CODE_2, SOC_CODE_4
df_data_statistics.loc[['SOC_CODE'],['preprocess_action','preprocess_comment','new_feature_name','new_feature_logic']]=['Drop column','Feature engineering','SOC_CODE_2,SOC_CODE_4','SOC_CODE.split(\'-\')']

In [73]:
#Feature engineering - EMPLOYER_COUNTRY - US or NOT
df_data_statistics.loc[['EMPLOYER_COUNTRY'],['preprocess_action','preprocess_comment','new_feature_name','new_feature_logic']]=['Drop column','Feature engineering','USA_YN','IF EMPLOYER_COUNTRY==USA THEN Y ELSE N END']

In [74]:
list(df_data_statistics.index)

['CASE_NUMBER',
 'CASE_STATUS',
 'RECEIVED_DATE',
 'DECISION_DATE',
 'ORIGINAL_CERT_DATE',
 'VISA_CLASS',
 'JOB_TITLE',
 'SOC_CODE',
 'SOC_TITLE',
 'FULL_TIME_POSITION',
 'BEGIN_DATE',
 'END_DATE',
 'TOTAL_WORKER_POSITIONS',
 'NEW_EMPLOYMENT',
 'CONTINUED_EMPLOYMENT',
 'CHANGE_PREVIOUS_EMPLOYMENT',
 'NEW_CONCURRENT_EMPLOYMENT',
 'CHANGE_EMPLOYER',
 'AMENDED_PETITION',
 'EMPLOYER_NAME',
 'TRADE_NAME_DBA',
 'EMPLOYER_ADDRESS1',
 'EMPLOYER_ADDRESS2',
 'EMPLOYER_CITY',
 'EMPLOYER_STATE',
 'EMPLOYER_POSTAL_CODE',
 'EMPLOYER_COUNTRY',
 'EMPLOYER_PROVINCE',
 'EMPLOYER_PHONE',
 'EMPLOYER_PHONE_EXT',
 'NAICS_CODE',
 'EMPLOYER_POC_LAST_NAME',
 'EMPLOYER_POC_FIRST_NAME',
 'EMPLOYER_POC_MIDDLE_NAME',
 'EMPLOYER_POC_JOB_TITLE',
 'EMPLOYER_POC_ADDRESS1',
 'EMPLOYER_POC_ADDRESS2',
 'EMPLOYER_POC_CITY',
 'EMPLOYER_POC_STATE',
 'EMPLOYER_POC_POSTAL_CODE',
 'EMPLOYER_POC_COUNTRY',
 'EMPLOYER_POC_PROVINCE',
 'EMPLOYER_POC_PHONE',
 'EMPLOYER_POC_PHONE_EXT',
 'EMPLOYER_POC_EMAIL',
 'AGENT_REPRESENTING_EMPL

In [81]:
#Drop columns - EMPLOYER_* except 'EMPLOYER_NAME',EMPLOYER_POSTAL_CODE
emp_cols=['TRADE_NAME_DBA','EMPLOYER_ADDRESS1','EMPLOYER_ADDRESS2','EMPLOYER_CITY','EMPLOYER_STATE',
          'EMPLOYER_COUNTRY','EMPLOYER_PROVINCE','EMPLOYER_PHONE','EMPLOYER_PHONE_EXT','EMPLOYER_POC_LAST_NAME',
          'EMPLOYER_POC_FIRST_NAME','EMPLOYER_POC_MIDDLE_NAME','EMPLOYER_POC_JOB_TITLE','EMPLOYER_POC_ADDRESS1',
          'EMPLOYER_POC_ADDRESS2','EMPLOYER_POC_CITY','EMPLOYER_POC_STATE','EMPLOYER_POC_POSTAL_CODE',
          'EMPLOYER_POC_COUNTRY','EMPLOYER_POC_PROVINCE','EMPLOYER_POC_PHONE','EMPLOYER_POC_PHONE_EXT','EMPLOYER_POC_EMAIL']
df_data_statistics.loc[emp_cols,['preprocess_action','preprocess_comment']]=['Drop column','Not Useful']


In [87]:
#Drop columns - AGENT_* AGENT_REPRESENTING_EMPLOYER
agt_cols=['AGENT_ATTORNEY_LAST_NAME','AGENT_ATTORNEY_FIRST_NAME','AGENT_ATTORNEY_MIDDLE_NAME','AGENT_ATTORNEY_ADDRESS1',
          'AGENT_ATTORNEY_ADDRESS2','AGENT_ATTORNEY_CITY','AGENT_ATTORNEY_STATE','AGENT_ATTORNEY_POSTAL_CODE',
          'AGENT_ATTORNEY_COUNTRY','AGENT_ATTORNEY_PROVINCE','AGENT_ATTORNEY_PHONE','AGENT_ATTORNEY_PHONE_EXT',
          'AGENT_ATTORNEY_EMAIL_ADDRESS','LAWFIRM_NAME_BUSINESS_NAME','STATE_OF_HIGHEST_COURT','NAME_OF_HIGHEST_STATE_COURT'      
]
df_data_statistics.loc[agt_cols,['preprocess_action','preprocess_comment']]=['Drop column','Not Useful']


In [88]:
df_data_statistics.loc[['AGENT_ATTORNEY_LAST_NAME','AGENT_ATTORNEY_FIRST_NAME','AGENT_ATTORNEY_MIDDLE_NAME','AGENT_ATTORNEY_ADDRESS1','AGENT_ATTORNEY_ADDRESS2','AGENT_ATTORNEY_CITY','AGENT_ATTORNEY_STATE','AGENT_ATTORNEY_POSTAL_CODE','AGENT_ATTORNEY_COUNTRY','AGENT_ATTORNEY_PROVINCE','AGENT_ATTORNEY_PHONE','AGENT_ATTORNEY_PHONE_EXT','AGENT_ATTORNEY_EMAIL_ADDRESS','LAWFIRM_NAME_BUSINESS_NAME','STATE_OF_HIGHEST_COURT','NAME_OF_HIGHEST_STATE_COURT']]

,Dtype,percent_missing,cardinality,preprocess_action,preprocess_comment,new_feature_name,new_feature_logic
AGENT_ATTORNEY_LAST_NAME,object,25.4611,97.394,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_FIRST_NAME,object,25.4611,98.1473,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_MIDDLE_NAME,object,58.8333,98.823,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_ADDRESS1,object,25.4611,96.801,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_ADDRESS2,object,41.5167,95.9745,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_CITY,object,25.4611,99.2658,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_STATE,object,25.4611,99.9637,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_POSTAL_CODE,object,25.4611,98.5723,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_COUNTRY,object,25.4611,99.9968,Drop column,Not Useful,NaN,NaN
AGENT_ATTORNEY_PROVINCE,object,96.9918,99.9612,Drop column,Not Useful,NaN,NaN


In [11]:
#add observations
dict_notes['drop_missing']=list(df_data_statistics[df_data_statistics.percent_missing>=40.0].index)
dict_notes['drop_high_cardinal']=list(df_data_statistics[df_data_statistics.cardinality<80.0].index)
dict_notes['numeric_columns']=list(df_data_statistics[df_data_statistics.Dtype=='float64'].index)
dict_notes['date_columns']=list(df_data_statistics[df_data_statistics.Dtype=='<M8[ns]'].index)
dict_notes['string_columns']=list(df_data_statistics[df_data_statistics.Dtype=='O'].index)

dict_notes['preprocess_pipeline']='Seperate target column->feature engineering->drop columns->impute missing values->encode non-numeric columns->scale/normalize->reset datatype'

In [27]:
data.groupby(['VISA_CLASS','CASE_STATUS']).size()

VISA_CLASS       CASE_STATUS          
E-3 Australian   Certified                  2578
                 Certified - Withdrawn        73
                 Denied                      139
                 Withdrawn                    87
H-1B             Certified                146932
                 Certified - Withdrawn      2802
                 Denied                     1017
                 Withdrawn                  2993
H-1B1 Chile      Certified                   232
                 Certified - Withdrawn         4
                 Denied                       18
                 Withdrawn                     6
H-1B1 Singapore  Certified                   269
                 Certified - Withdrawn         5
                 Denied                       12
                 Withdrawn                     6
dtype: int64

In [30]:
#add observations
dict_notes['feature_eng']='1.PROCESSING_DAYS=Decision_date-Received_date 2.APPROVAL_VALIDITY=End_date-Begin_date 3.WAGE_RATE_OF_PAY_TO - impute with WAGE_RATE_OF_PAY_FROM'

In [19]:
dict_notes.keys()

dict_keys(['output_class', 'drop_missing', 'drop_high_cardinal', 'numeric_columns', 'date_columns', 'string_columns', 'feature_eng', 'preprocess_pipeline'])

In [31]:
dict_notes['feature_eng']

'1.PROCESSING_DAYS=Decision_date-Received_date\n 2.APPROVAL_VALIDITY=End_date-Begin_date\n 3.WAGE_RATE_OF_PAY_TO - impute with WAGE_RATE_OF_PAY_FROM'